In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from minimizers.minimize import CustomLossMinimizer, CustomLossClassifier

In [2]:
seed = 0

---

In [3]:
X, y = make_classification(
    n_classes=2, 
    n_samples=1000, 
    n_features=10, 
    flip_y=0., 
    n_clusters_per_class=1, 
    random_state=seed+1,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
del X, y

In [4]:
mod = Pipeline([
    ('sc', StandardScaler()),
    ('clf', CustomLossMinimizer(random_state=seed)),
])

In [5]:
mod.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('clf', CustomLossMinimizer(random_state=0))])

In [6]:
y_hat = mod.predict(X_train).round()
(y_hat==y_train).mean()

0.8946666666666667

In [7]:
y_hat = mod.predict(X_test).round()
(y_hat==y_test).mean()

0.916

---

In [8]:
X, y = make_classification(
    n_classes=3, 
    n_samples=1000, 
    n_features=10, 
    flip_y=0., 
    n_clusters_per_class=1, 
    random_state=seed+2,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
del X, y

In [9]:
mod = Pipeline([
    ('sc', StandardScaler()),
    ('clf', CustomLossClassifier(random_state=seed)),
])

In [10]:
mod.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('clf', CustomLossClassifier(random_state=0))])

In [18]:
y_hat = mod.predict(X_train).argmax(1)
(y_hat==y_train).mean()

0.8493333333333334

In [20]:
y_hat = mod.predict(X_test).argmax(1)
(y_hat==y_test).mean()

0.848

---